In [1]:
%pylab inline

from scipy.optimize import root_scalar
from scipy.integrate import trapezoid

Populating the interactive namespace from numpy and matplotlib


In formalism.ipynb we investigated the basic outline of the paper. In this notebook we will extend the method a little bit, as noted in the conclusion. One of the ways proposed to improve this measurement of $\pi$ is to use more than a single half period of the probability function to estimate $\pi$. In this notebook I'll introduce some code that uses two half-periods per qubit to estimate $\pi$.

In [2]:
class Qubit:
    def __init__(self, seed, alpha, beta, phi0, c):
        self.alpha = alpha
        self.beta = beta
        self.phi0 = phi0
        self.c = c
        
        self.prob = lambda t: alpha / 2 * (1 - np.cos(c * t + phi0)) + beta
        
        self.rng = np.random.default_rng(seed)
        
    def measure(self, t, n=1):
        # The n parameter allows us to run multiple measurements for the same time step
        # with one function call. This speeds up the computation a bit.
        v = self.rng.random(size=n)
        p = self.prob(t)
        
        # "less than" ensures the probability actually works as intended
        return np.where(v < p, 1, 0)

Must of this following code remains the same as before, so I've kept it the same up to the point where things change, at which point I'll introduce some commentary. The first obvious change is that we're going to need to get to at least $3\pi$ in time steps, so I'll just call it an even 10.

In [5]:
time_steps = np.arange(0, 10, 0.1)

q = {}
f = {}
f1 = {}
f1_interp = {}
alpha_hat = {}
beta_hat = {}

for i in range(5):
    seed = 5 * i + 7 * (i % 3)
    print(f"Adding qubit with seed {seed}")
    q[i] = Qubit(seed, 1, 0, 0, 1)
    
    
    vals = []
    n_measure = 8192
    for t in time_steps:
        m = q[i].measure(t, n_measure)
        frac = np.sum(m) / len(m)
        vals.append(frac)

    f[i] = np.asarray(vals)

    beta_hat[i] = np.min(f[i])
    alpha_hat[i] = np.max(f[i]) - beta_hat[i]

    f1[i] = (f[i] - beta_hat[i]) / alpha_hat[i]
    f1_interp[i] = lambda x: np.interp(x, time_steps, f1[i])

    print(alpha_hat[i], beta_hat[i])

Adding qubit with seed 0
0.9998779296875 0.0
Adding qubit with seed 12
0.999755859375 0.0
Adding qubit with seed 24
1.0 0.0
Adding qubit with seed 15
0.9998779296875 0.0
Adding qubit with seed 27
0.999755859375 0.0


As before, Nwe want to find where $\tilde{f_1} \approx 0.5$, except now instead of being just close to $t_1 = 1.5$ and $t_2 = 4.5$ we also seek where it's close to $t_3=8$. As before I define an $f_2$ interpolation that is simply $f_1 - 0.5$ for root finding purposes, since the root finder looks for 0's.

In [9]:
t1 = {}
t2 = {}
t3 = {}

for i in range(len(q)):
    f2 = lambda x: np.interp(x, time_steps, f1[i]) - 0.5

    t1[i] = root_scalar(f2, x0=1.5, x1=2).root
    t2[i] = root_scalar(f2, x0=4.5, x1=5).root
    t3[i] = root_scalar(f2, x0=7.6, x1=8).root

    print(t1[i], t2[i], t3[i])

1.5726506024096387 4.690212264150944 7.861341463414635
1.573396674584323 4.726566416040101 7.855621301775149
1.5894021739130435 4.7131336405529956 7.856474820143886
1.5715533980582526 4.7022300469483564 7.864319809069213
1.5818791946308726 4.691688311688312 7.855480984340044


Next steps are the same as before, where we estimate and refine $\hat{\alpha}$ and $\hat{\beta}$, except this time we no longer have to vaguely estimate $t_{minval}$ from $t_1$ and $t_2$, this time we can know it must be between $t_2$ and $t_3$.

In [13]:
delta = 0.1
t1_hat = {}
t2_hat = {}
t3_hat = {}

for i in range(len(q)):
    t_maxval = (t1[i] + t2[i]) / 2
    t_minval = (t2[i] + t3[i]) / 2

    # Indices of time steps that satisfy this condition
    tmax = np.where(np.abs((time_steps - t_maxval)) < delta)[0]
    tmin = np.where(np.abs((time_steps - t_minval)) < delta)[0]

    beta_hat[i] = np.mean(f1[i][tmin])
    alpha_hat[i] = np.mean(f1[i][tmax]) - beta_hat[i]

    f1[i] = (f[i] - beta_hat[i]) / alpha_hat[i]
    f1_interp[i] = lambda x: np.interp(x, time_steps, f1[i])

    print(alpha_hat[i], beta_hat[i])
    
    # Indices of time steps that fulfill the above condition.
    t1_idx = np.where(np.abs((time_steps - t1[i])) < 0.5)[0]
    coeff_1 = np.polyfit(time_steps[t1_idx], f1[i][t1_idx], 1)

    # Once again subtracting 0.5 to find where the function = 0.5 not 0.
    fit_1 = lambda x: coeff_1[0] * x + coeff_1[1] - 0.5
    t1_hat[i] = root_scalar(fit_1, x0=1.5, x1=2).root

    # Indices of time steps that fulfill the above condition.
    t2_idx = np.where(np.abs((time_steps - t2[i])) < 0.5)[0]
    coeff_2 = np.polyfit(time_steps[t2_idx], f1[i][t2_idx], 1)

    # Once again subtracting 0.5 to find where the function = 0.5 not 0.
    fit_2 = lambda x: coeff_2[0] * x + coeff_2[1] - 0.5
    t2_hat[i] = root_scalar(fit_2, x0=4.5, x1=5).root
    
    # Indices of time steps that fulfill the above condition.
    t3_idx = np.where(np.abs((time_steps - t3[i])) < 0.5)[0]
    coeff_3 = np.polyfit(time_steps[t3_idx], f1[i][t3_idx], 1)

    # Once again subtracting 0.5 to find where the function = 0.5 not 0.
    fit_3 = lambda x: coeff_3[0] * x + coeff_3[1] - 0.5
    t3_hat[i] = root_scalar(fit_3, x0=7.5, x1=8).root


    print("New times", t1_hat[i], t2_hat[i], t3_hat[i])

0.9998779296875 -2.3900922930516827e-07
New times 1.5648532188189983 4.703886542866155 7.856003984615389
0.9997558593749999 -6.577580254126664e-07
New times 1.5695339742117738 4.716952338985774 7.855642278423961
1.0 0.0
New times 1.5769093482675858 4.713788798816388 7.854594163418265
0.9998779296875001 -1.4933967069257533e-07
New times 1.5696075355981445 4.709468270668766 7.850838258702262
0.999755859375 -2.6876211019433377e-07
New times 1.5752135393827447 4.707112017461871 7.854356267390136


Finaly we estimate the integral between $t_1$ and $t_2$ using the trapezoidal rule, which then gives us an estimate of $\pi$ via $\pi \approx \frac{t_2 - t_1}{I}$. This is the same as before, of course. However we must now include an additional step, and estimate the integral between $t_2$ and $t_3$ in order to get a second estimate from that time period. 

In [17]:
pi_guess_t1t2 = {}
pi_guess_t2t3 = {}
for i in range(len(q)):
    f_hat = lambda x: np.interp(x, time_steps, f[i])

    # Adding the t1 and t2 to the measured timesteps for integration purposes.
    t_idx = np.where((time_steps >= t1[i]) & (time_steps <= t2[i]))[0]
    t_integ = time_steps[t_idx]
    t_integ = np.insert(t_integ, 0, t1_hat[i])
    t_integ = np.append(t_integ, t2_hat[i])

    # Doing the same for the probability measured values
    y_integ = f[i][t_idx]
    y_integ = np.insert(y_integ, 0, f_hat(t1_hat[i]))
    y_integ = np.append(y_integ, f_hat(t2_hat[i]))

    # Trapezoidal rule on the given points
    I = np.trapz(y_integ - 0.5, t_integ)

    # And the final guess!
    pi_guess_t1t2[i] = (t2_hat[i] - t1_hat[i]) / I
    
    
    # Adding the t2 and t3 to the measured timesteps for integration purposes.
    t_idx = np.where((time_steps >= t2[i]) & (time_steps <= t3[i]))[0]
    t_integ = time_steps[t_idx]
    t_integ = np.insert(t_integ, 0, t2_hat[i])
    t_integ = np.append(t_integ, t3_hat[i])

    # Doing the same for the probability measured values
    y_integ = f[i][t_idx]
    y_integ = np.insert(y_integ, 0, f_hat(t2_hat[i]))
    y_integ = np.append(y_integ, f_hat(t3_hat[i]))

    # Trapezoidal rule on the given points
    # We expect this one to be negative, so to estimate pi we will invert it to positve.
    I = -np.trapz(y_integ - 0.5, t_integ)

    # And the final guess!
    pi_guess_t1t2[i] = (t2_hat[i] - t1_hat[i]) / I
    pi_guess_t2t3[i] = (t3_hat[i] - t2_hat[i]) / I

In [19]:
pi_t1t2 = np.nanmean(list(pi_guess_t1t2.values()))
pi_t2t3 = np.nanmean(list(pi_guess_t2t3.values()))

pi_t1t2, pi_t2t3, np.mean((pi_t1t2, pi_t2t3))

(3.139480440972794, 3.1445000131483054, 3.1419902270605498)

Even extending the method in this simple way already massively improves our accuracy, although keep in mind again that we're working with "ideal" qubits, which have a "perfect" probability distribution for the $\lvert 1 \rangle$ state.